In [2]:
import numpy as np
import time as tm
import warnings
warnings.filterwarnings("ignore")
from sklearn.svm import LinearSVC

avg_sum = 0.0
len_sum = 0
N = 16

myVars = globals()
global name 
name = "dataset"
global model

model = [[None for i in range(N)] for j in range(N)]

for i in range(0,16):
    for j in range (i+1, 16):
        myVars.__setitem__(name + str(i) + str(j) + str(0), np.zeros((1, 129)))
        myVars.__setitem__(name + str(i) + str(j) + str(1), np.zeros((1, 129)))

def createFeatures(Z_train):
    X = Z_train[:,:64]
    Y = Z_train[:,-1:]
    return np.concatenate((X, 1 - X, Y), axis=1) 

def createAandB(dataset):
    X = 8*dataset[:, 64] + 4*dataset[:, 65] + 2*dataset[:, 66] + dataset[:, 67]
    Y = 8*dataset[:, 68] + 4*dataset[:, 69] + 2*dataset[:, 70] + dataset[:, 71]

    X = np.reshape(X, (len(X), 1))
    Y = np.reshape(Y, (len(Y), 1))
    return np.concatenate((X, Y), axis=1)


def cleanData(dataset, AB, testOrTrain):
    AB = AB.astype(int)
    isLarger = AB[:,0] > AB[:, 1]
    isLarger = np.reshape(isLarger, (len(isLarger), 1))
    isLarger = isLarger.astype(int)
    dataset[:,-1] = np.bitwise_xor(isLarger[:, -1],(dataset[:, -1].astype(int)))

    isLarger = AB[:,0] > AB[:, 1]
    copyMatrix = np.copy(AB[:, 0])
    AB[isLarger, 0] = AB[isLarger, 1]
    AB[isLarger, 1] = copyMatrix[isLarger]

    for index in range (1, len(dataset)):
        if(AB[index, 0] == AB[index, 1]):
            continue        
        myVars.__setitem__(name + str(AB[index, 0]) + str(AB[index, 1]) + str(testOrTrain), 
                           np.concatenate((myVars.__getitem__(name + str(AB[index, 0]) + str(AB[index, 1]) + str(testOrTrain)), 
                                          np.reshape(dataset[index,:], (1, len(dataset[index, :])))),axis=0))
    
    for i in range (0, N):
        for j in range (i+1, N):
            myVars.__setitem__(name + str(i) + str(j) + str(testOrTrain), 
                           np.delete(myVars.__getitem__(name + str(i) + str(j) + str(testOrTrain)), (0), axis=0))
            
    return 

def transformData(Z_train, testOrTrain):
    train_features = createFeatures(Z_train)
    AB_train = createAandB(Z_train)
    cleanData(train_features, AB_train, testOrTrain)
    return

def my_fit(Z_train):

    transformData(Z_train, 0)
    
    for i in range (0, N):
        for j in range (i+1, N):
            train_ = myVars.__getitem__(name + str(i) + str(j) + str(0))
            model[i][j] = training(train_)
            
    return model

def training(data):
    clf = LinearSVC(loss = "hinge")
    clf.fit(data[:,:-1], data[:,-1])
    return clf

 
def my_predict(Z_tst, model):
    global avg_sum
    AB_train = createAandB(Z_tst).astype(int)
    new_tst = np.concatenate((Z_tst[:,:64], 1-Z_tst[:,:64]), axis = 1)
    y_predict = np.zeros((len(Z_tst),1))
    
    for index in range(0, len(Z_tst)):
        a = AB_train[index, 0]
        b = AB_train[index, 1]
        
        if(a>b):
            y_predict[index,0] = 1 - model[b][a].predict(np.reshape(new_tst[index,:],(1, len(new_tst[index]))))
        else:
            y_predict[index,0] = model[a][b].predict(np.reshape(new_tst[index,:],(1, len(new_tst[index]))))
    avg_sum = np.sum((np.reshape(y_predict,((len(y_predict),1)))  == np.reshape(Z_tst[:,-1], ((len(Z_tst),1)))).astype(int))
    return
            
            
            
Z_trn = np.loadtxt( "train.dat" )
Z_tst = np.loadtxt( "test.dat" )

start_time = tm.time()
my_predict(Z_tst, my_fit(Z_trn))
end_time = tm.time()

print(100*avg_sum/len(Z_tst))
print("time taken:", (end_time - start_time), "sec")

94.1875
time taken: 7.2340381145477295 sec
